In [1]:
# /Users/teliov/TUD/Thesis/Medvice/Notebooks/data/04_06_new_data/data/split
# So we can use the *thesislib* package
import sys
import os

module_path = os.path.abspath("..")

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
import pandas as pd

In [3]:
from thesislib.utils import pathutils

In [34]:
symptoms_csv = pathutils.get_data_file("04_06_new_data/data/symptoms_04_16_4.csv")

In [35]:
symptom_columns = ['PATIENT', 'GENDER', 'RACE', 'ETHNICITY', 'AGE_BEGIN', 'AGE_END',
                       'PATHOLOGY', 'NUM_SYMPTOMS', 'SYMPTOMS']
usecols = ['GENDER', 'RACE', 'AGE_BEGIN', 'PATHOLOGY', 'NUM_SYMPTOMS', 'SYMPTOMS']

In [36]:
df = pd.read_csv(symptoms_csv, usecols=usecols)

In [37]:
symptoms_db_json = pathutils.get_data_file("definitions/symptoms_db.json")
conditions_db_json = pathutils.get_data_file("definitions/conditions_db.json")

In [38]:
import json

with open(symptoms_db_json) as fp:
    symptoms_db = json.load(fp)

with open(conditions_db_json) as fp:
    conditions_db = json.load(fp)

In [39]:
condition_labels = {code: idx for idx, code in enumerate(sorted(conditions_db.keys()))}

In [40]:
df['LABEL'] = df.PATHOLOGY.apply(lambda v: condition_labels.get(v))

In [41]:
race_code = {'white': 0, 'black':1, 'asian':2, 'native':3, 'other':4}
df['RACE'] = df.RACE.apply(lambda v: race_code.get(v))

In [42]:
df['GENDER'] = df.GENDER.apply(lambda gender: 0 if gender == 'F' else 1)

In [43]:
df = df.rename(columns={'AGE_BEGIN': 'AGE'})

In [44]:
def _symptom_transform(val, labels):
    parts = val.split(";")
    res = ",".join([labels.get(item) for item in parts])
    return res

In [45]:
symptom_map  ={code: str(idx) for idx, code in enumerate(sorted(symptoms_db.keys()))}

In [46]:
df = df[df.NUM_SYMPTOMS > 0]

In [47]:
df['SYMPTOMS'] = df.SYMPTOMS.apply(_symptom_transform, labels=symptom_map)

In [48]:
ordered_keys = ['LABEL', 'GENDER', 'RACE', 'AGE', 'SYMPTOMS']
df = df[ordered_keys]

In [49]:
output_file = pathutils.get_data_file("04_06_new_data/sparse/symptoms_04_16_4.csv")

In [50]:
df.index.name = "Index"

In [51]:
df.to_csv(output_file)

In [33]:
# There was an almost 90% reduction in the size of the original file!

376